## 实验要求
### 截止日期：12月15日
作业的提交格式参考之前的说明，提交到18329300691@163.com
### 基本要求
a)	基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
b)	基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；
### 中级要求
a)  对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
b)	对测试集Watermelon-test2进行预测，输出分类精度；
### 高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。



## 什么是决策树
<img src="https://s2.loli.net/2022/10/23/yTpSLmgFWOh4Y5d.png" style="zoom:66%" align="left"/>


## 决策树的划分
- 决策树主要分为三种：
	ID3，C4.5和CART，它们分别对应的**特征选择准则**是信息增益（ID3），信息增益比（C4.5）和基尼指数（CART）。
	它们决定当前选择哪个特征进行数据划分，使得样本在当下能够被最大程度的划分。
- 对于离散变量，选定**属性**分类即可；
- 对于连续变量，需要选定**划分点**。
- CART和C4.5支持数据特征为**连续分布**时的处理，能够完成对连续属性的离散化处理，主要通过二元切分的方式来处理连续型变量，这个分裂点的选择原则是使得划分后的子树中的“混乱程度”降低。


## C4.5算法
- C4.5算法与ID3算法相似，其对ID3算法进行了改进。
- 信息增益作为划分准则存在的问题：

     信息增益偏向于选择取值较多的特征进行划分。⽐如学号这个特征，每个学生都有一个不同的学号，如果根据学号对样本进行分类，则每个学生都属于不同的类别，这样是没有意义的。而C4.5在生成过程中，用**信息增益比**来选择特征，可以校正这个问题。
     
- 特点
  - 能够完成对连续属性的离散化处理
  - 能够对不完整数据进行处理
  - 需要对数据集进行多次的顺序扫描和排序



## CART算法
- ID3和C4.5虽然在对训练样本集的学习中可以尽可能多的挖掘信息，但其生成的决策树分支较大，规模较大。为了简化决策树的规模，提高生成决策树的效率，就出现了根据**基尼指数**来选择的CART； 
- 对于给定的样本集合 ，其基尼指数为： $$ {Gini}(D)=1-\sum_{k=1}^{K}\left(\frac{\left|C_{k}\right|}{|D|}\right)^{2} $$
   其中$𝐶_𝑘$是𝐷中属于第𝑘类的样本子集，K是类的个数。
- 基尼系数的性质与信息熵一样：
   度量随机变量的不确定度的大小；基尼指数越⼩表示数据的纯度越高，反之其值越大，样本集合的不确定性也就越大。


## 决策树的剪枝
- 决策树很容易出现**过拟合现象**。原因在于学习时完全考虑的是如何提⾼对训练数据的正确分类从⽽构建出过于复杂的决策树。
- 解决这个问题的方法称为**剪枝**，即对已生成的树进行简化。具体地，就是从已生成的树上裁剪掉⼀些子树或叶节点，并将其根节点或父节点作为新的叶节点。 
- 决策树的剪枝基本策略有**预剪枝 (Pre-Pruning)** 和 **后剪枝 (Post-Pruning)**
   - **预剪枝**：是根据⼀些原则**极早的停止树增长**，如树的深度达到用户所要的深度、节点中样本个数少于用户指定个数、不纯度指标下降的幅度小于用户指定的幅度等。 
   - **后剪枝**：是通过在完全生长的树上剪去分枝实现的，通过删除节点的分支来剪去树节点。是在生成决策树之后**自底向上**的对树中所有的非叶结点进⾏逐一考察 。

### 基本要求
a)	基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；

b)	基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；

In [85]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

In [5]:
# 读取训练集和测试集数据
watermelon_train1_data = pd.read_csv(r"C:/Users/LENOVO/Desktop/ML-6/Watermelon-train1.csv", encoding='gbk')  
watermelon_test1_data = pd.read_csv(r"C:/Users/LENOVO/Desktop/ML-6/Watermelon-test1.csv", encoding='gbk')

In [6]:
# 将分类标签转换为数值表示，是->1，否->0
watermelon_train1_data['好瓜'] = watermelon_train1_data['好瓜'].map({'是': 1, '否': 0})
watermelon_test1_data['好瓜'] = watermelon_test1_data['好瓜'].map({'是': 1, '否': 0})

In [7]:
# 对离散特征进行独热编码
watermelon_train1_data = pd.get_dummies(watermelon_train1_data, columns=['色泽', '根蒂', '敲声', '纹理'])
watermelon_test1_data = pd.get_dummies(watermelon_test1_data, columns=['色泽', '根蒂', '敲声', '纹理'])

In [8]:
# 提取特征和标签
X_train = watermelon_train1_data.drop('好瓜', axis=1)
y_train = watermelon_train1_data['好瓜']

X_test = watermelon_test1_data.drop('好瓜', axis=1)
y_test = watermelon_test1_data['好瓜']


## ID3算法
- ID3算法的核⼼思想应用信息增益准则作为标准,介绍信息增益之前首先介绍一下信息熵和条件熵： 
- 熵（entropy）概念：
	    1948年，香农提出了“信息熵”的概念。在信息论与概率统计中，熵是表示随机变量不确定性的量。X是⼀个取值为有限个的离散随机变量，
$$ H(X)=-\sum_{i=1}^{n} p\left(x_{i}\right) \log p\left(x_{i}\right)$$ 
$𝐻(𝑋)$就被称作随机变量𝑋的熵，它表示随机变量不确定的度量。熵取值越大，随机变量不确定性越大。当随机变量为均匀分布时，熵最大。当某一状态概率取值为1时，熵的值为零。

### ID3算法-条件熵和信息增益
- 条件熵 $𝐻(𝑌∣𝑋)$ ：
	表示在已知随机变量𝑋的条件下随机变量𝑌的不确定性，定义为给定𝑋条件下𝑌的条件概率分布的熵对𝑋的数学期望:
$$H(Y \mid X)=\sum_{x} p(x) H(Y \mid X=x) =-\sum_{x} p(x) \sum_{y} p(y \mid x) \log p(y \mid x)$$

- 特征𝐴对数据集𝐷的信息增益就是熵$𝐻(𝐷)$与条件熵$𝐻(𝐷|𝐴)$之差:
$$𝐻(𝐷)−𝐻(𝐷∣𝐴)$$

	表示已知特征𝐴的信息而使得数据集𝐷的信息不确定减少的程度。信息增益越大的特征代表其具有更强的分类能力，所以我们就要**选择能够使数据的不确定程度减少最多的特征**，也就是信息增益最大的特征。

### ID3算法-停止条件
- 决策树的生成:

	从根节点开始，计算所有可能特征的信息增益，选择信息增益最大的特征作为划分该节点的特征，根据该特征的不同取值建立子节点；
	在对子节点递归地调用以上方法，直到达到停止条件，得到⼀个决策树。
    
    
- 迭代停止条件：
  1. 当前结点所有样本都属于同⼀类别；
  2. 当前结点的所有属性值都相同，没有剩余属性可用来进一步划分样本；
  3. 达到最大树深；
  4. 达到叶子结点的最小样本数；

### ID3算法举例

<img src="https://s2.loli.net/2022/10/23/p7gSQeYGnoBCd2i.png" style="zoom:64%" />


$$
\begin{array}{l}
\operatorname{Info}^{\text {In }}(D)=-\frac{9}{14} \log _{2}\left(\frac{9}{14}\right)-\frac{5}{14} \log _{2}\left(\frac{5}{14}\right)=0.940 \\
\operatorname{Infoage~}(D)=\frac{5}{14} \times\left(-\frac{2}{5} \times \log _{2} \frac{2}{5}-\frac{3}{5} \times \log _{2} \frac{3}{5}\right)+\frac{4}{14} \times\left(-\frac{4}{4} \times \log _{2} \frac{4}{4}-\frac{0}{4} \times \log _{2} \frac{0}{4}\right) 
+\frac{5}{14} \times\left(-\frac{2}{5} \times \log _{2} \frac{2}{5}-\frac{3}{5} \times \log _{2} \frac{3}{5}\right)=0.694 \\
\text { Gain }(\text { age })=\operatorname{Info}(D)-\operatorname{InfO}_{\text {age }}(D) =0.940-0.694=0.246
\end{array}
$$

<img src="https://s2.loli.net/2022/10/23/1zAnHWKRgQ9FaJV.png" style="zoom:72%" />
类似地，
Gain(income)=0.029    
Gain(student)=0.151    
Gain(credit_rating)=0.048

所以，选择age作为第一个根节点



In [9]:
# 构建 ID3 决策树模型
dt_classifier = DecisionTreeClassifier(criterion='entropy')
dt_classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [12]:
# 对测试集进行预测
y_pred = dt_classifier.predict(X_test)

# 输出分类精度
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"分类精度：{accuracy:.2f}")

分类精度：0.70


In [13]:
class Node:
    def __init__(self, feature=None, value=None, result=None):
        self.feature = feature  # 特征列名
        self.value = value  # 特征取值
        self.result = result  # 结果标签
        self.children = {}  # 子节点

X是⼀个取值为有限个的离散随机变量，
$$ H(X)=-\sum_{i=1}^{n} p\left(x_{i}\right) \log p\left(x_{i}\right)$$ 
$𝐻(𝑋)$就被称作随机变量𝑋的熵，它表示随机变量不确定的度量。熵取值越大，随机变量不确定性越大。当随机变量为均匀分布时，熵最大。当某一状态概率取值为1时，熵的值为零。

In [14]:
# 计算熵
def entropy(y):
    unique_labels, counts = np.unique(y, return_counts=True)# 获取类别标签及其对应的出现次数
    probabilities = counts / len(y)# 计算每个类别的概率
    return -np.sum(probabilities * np.log2(probabilities + 1e-10))

- 条件熵 $𝐻(𝑌∣𝑋)$ ：
	表示在已知随机变量𝑋的条件下随机变量𝑌的不确定性，定义为给定𝑋条件下𝑌的条件概率分布的熵对𝑋的数学期望:
$$H(Y \mid X)=\sum_{x} p(x) H(Y \mid X=x) =-\sum_{x} p(x) \sum_{y} p(y \mid x) \log p(y \mid x)$$

- 特征𝐴对数据集𝐷的信息增益就是熵$𝐻(𝐷)$与条件熵$𝐻(𝐷|𝐴)$之差:
$$𝐻(𝐷)−𝐻(𝐷∣𝐴)$$

In [15]:
# 计算信息增益
def information_gain(X, y, feature):
    # 计算基础熵（即未进行任何特征划分时的熵）
    base_entropy = entropy(y)
    # 获取特征列的唯一值
    unique_values = X[feature].unique()
    # 初始化加权熵
    weighted_entropy = 0
    # 遍历特征的每个取值
    for value in unique_values:
         # 获取特征X取值为value的样本的索引
        subset_indices = X[X[feature] == value].index
          # 计算在特征 X 取值为 value 的条件下目标变量 y 的熵
        subset_entropy = entropy(y.loc[subset_indices])
         # 将条件熵加权累加
        weighted_entropy += len(subset_indices) / len(y) * subset_entropy
 # 计算信息增益
    return base_entropy - weighted_entropy


- 决策树的生成:

	从根节点开始，计算所有可能特征的信息增益，选择信息增益最大的特征作为划分该节点的特征，根据该特征的不同取值建立子节点；
	在对子节点递归地调用以上方法，直到达到停止条件，得到⼀个决策树。
    
    
- 迭代停止条件：
  1. 当前结点所有样本都属于同⼀类别；
  2. 当前结点的所有属性值都相同，没有剩余属性可用来进一步划分样本；
  3. 达到最大树深；
  4. 达到叶子结点的最小样本数；

In [16]:
# 构建ID3决策树
def id3(X, y, features):   
    # 如果目标变量y的取值只有一个，返回一个叶子节点，表示该子树的预测结果为该唯一取值
    if len(set(y)) == 1:
        return Node(result=y.iloc[0])

    # 如果特征集为空，表示所有特征都已经使用过。没有剩余属性可用来进一步划分样本，停止迭代
    # 返回一个叶子节点，表示该子树的预测结果为目标变量y中出现次数最多的取值
    if len(features) == 0:
        return Node(result=y.mode().iloc[0])

    # 选择信息增益最大的特征作为当前节点的划分特征
    best_feature = max(features, key=lambda feature: information_gain(X, y, feature))
    root = Node(feature=best_feature)

     # 对于划分特征的每个取值，递归构建子树
    for value in X[best_feature].unique():
        # 找到划分特征取值为value的样本索引
        subset_indices = X[X[best_feature] == value].index
        # 递归构建子树
        child_node = id3(X.loc[subset_indices], y.loc[subset_indices], features - {best_feature})
         # 将子树加入当前节点的子节点字典
        root.children[value] = child_node

    return root

In [17]:
# 预测单个样本
def predict(node, sample):   
    if node.result is not None:
        return node.result

    value = sample[node.feature]
    if value in node.children:
        return predict(node.children[value], sample)
    else:
        # 如果测试集中出现了训练集中未见过的取值，返回一个默认值
        return node.children[list(node.children.keys())[0]].result

In [18]:
# 批量预测
def batch_predict(node, X):    
    return X.apply(lambda sample: predict(node, sample), axis=1)

In [19]:
# 提取特征和标签
X_train = watermelon_train1_data.drop('好瓜', axis=1)
y_train = watermelon_train1_data['好瓜']

X_test = watermelon_test1_data.drop('好瓜', axis=1)
y_test = watermelon_test1_data['好瓜']

In [20]:
# 获取特征集合
features = set(X_train.columns)

In [21]:
# 构建ID3决策树
root = id3(X_train, y_train, features)

In [23]:
# 对测试集进行预测
y_pred = batch_predict(root, X_test)

# 输出分类精度
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"分类精度：{accuracy: .2f}")

分类精度： 0.70


### 中级要求
a)  对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；

b)	对测试集Watermelon-test2进行预测，输出分类精度；

In [25]:
# 读取训练集和测试集数据
watermelon_train2_data = pd.read_csv(r"C:/Users/LENOVO/Desktop/ML-6/Watermelon-train2.csv", encoding='gbk')  
watermelon_test2_data = pd.read_csv(r"C:/Users/LENOVO/Desktop/ML-6/Watermelon-test2.csv", encoding='gbk')

In [26]:
# 将分类标签转换为数值表示，是->1，否->0
watermelon_train2_data['好瓜'] = watermelon_train2_data['好瓜'].map({'是': 1, '否': 0})
watermelon_test2_data['好瓜'] = watermelon_test2_data['好瓜'].map({'是': 1, '否': 0})

In [27]:
# 将类别型特征转换为数字
le = LabelEncoder()
for column in watermelon_train2_data.columns:
    if watermelon_train2_data[column].dtype == 'object':
        watermelon_train2_data[column] = le.fit_transform(watermelon_train2_data[column])
        
for column in watermelon_test2_data.columns:
    if watermelon_test2_data[column].dtype == 'object':
        watermelon_test2_data[column] = le.fit_transform(watermelon_test2_data[column])

In [28]:
# 进行独热编码
# watermelon_train2_data = pd.get_dummies(watermelon_train2_data, columns=['色泽', '根蒂', '敲声', '纹理', '密度'])
# watermelon_test2_data = pd.get_dummies(watermelon_test2_data, columns=['色泽', '根蒂', '敲声', '纹理','密度'])

In [29]:
# 分割训练集特征和标签
X_train = watermelon_train2_data.drop(columns=['好瓜'])
y_train = watermelon_train2_data ['好瓜']

# 分割测试集特征和标签
X_test = watermelon_test2_data.drop(columns=['好瓜'])
y_test = watermelon_test2_data['好瓜']


### 决策树的划分
- 决策树主要分为三种：
	ID3，C4.5和CART，它们分别对应的**特征选择准则**是信息增益（ID3），信息增益比（C4.5）和基尼指数（CART）。
	它们决定当前选择哪个特征进行数据划分，使得样本在当下能够被最大程度的划分。
- 对于离散变量，选定**属性**分类即可；
- 对于连续变量，需要选定**划分点**。
- CART和C4.5支持数据特征为**连续分布**时的处理，能够完成对连续属性的离散化处理，主要通过二元切分的方式来处理连续型变量，这个分裂点的选择原则是使得划分后的子树中的“混乱程度”降低。

In [30]:
# 构建C4.5决策树
c4_5_classifier = DecisionTreeClassifier(criterion='entropy')
c4_5_classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [35]:
# 预测并输出分类精度
y_pred_c4_5 = c4_5_classifier.predict(X_test)
accuracy_c4_5 = accuracy_score(y_test, y_pred_c4_5)
print(f"C4.5 Classification Accuracy: {accuracy_c4_5:.2f}")

C4.5 Classification Accuracy: 0.60


In [32]:
pip install c45

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement c45 (from versions: none)
ERROR: No matching distribution found for c45


In [33]:
# 构建CART决策树
cart_classifier = DecisionTreeClassifier(criterion='gini')
cart_classifier.fit(X_train, y_train)

DecisionTreeClassifier()

In [36]:
# 预测并输出分类精度
y_pred_cart = cart_classifier.predict(X_test)
accuracy_cart = accuracy_score(y_test, y_pred_cart)
print(f"CART Classification Accuracy: {accuracy_cart:.2f}")

CART Classification Accuracy: 0.60


### C4.5算法
- C4.5算法与ID3算法相似，其对ID3算法进行了改进。
- 信息增益作为划分准则存在的问题：

     信息增益偏向于选择取值较多的特征进行划分。⽐如学号这个特征，每个学生都有一个不同的学号，如果根据学号对样本进行分类，则每个学生都属于不同的类别，这样是没有意义的。而C4.5在生成过程中，用**信息增益比**来选择特征，可以校正这个问题。
     
- 特点
  - 能够完成对连续属性的离散化处理
  - 能够对不完整数据进行处理
  - 需要对数据集进行多次的顺序扫描和排序


In [37]:
class C45DecisionTree:
    
    """
    min_samples_split: int, optional (default=2)
          节点分裂的最小样本数
    max_depth: int or None, optional (default=None)
          树的最大深度。如果为None，则节点会一直分裂，直到每个叶子节点的样本数小于min_samples_split     
    tree: dict
          存储构建好的决策树的字典    
    """
    
    def __init__(self, min_samples_split=2, max_depth=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.tree = None

    # 计算信息熵
    def _calculate_entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))
    
    # 根据训练数据拟合C4.5决策树
    def fit(self, X, y):
        features = X.columns
        data = pd.concat([X, y], axis=1)
        self.tree = self._build_tree(data, features)

    # 递归构建C4.5决策树
    def _build_tree(self, data, features, depth=0):
        # 若样本数量小于等于阈值或深度达到最大深度，则返回叶子节点
        if len(data) <= self.min_samples_split or (self.max_depth is not None and depth == self.max_depth):
            return self._create_leaf_node(data)

        # 选择最佳特征和阈值进行分裂
        selected_feature, threshold = self._choose_best_feature(data, features)

        # 根据选择的特征和阈值进行分裂
        left_data = data[data[selected_feature] <= threshold]
        right_data = data[data[selected_feature] > threshold]

        # 递归构建左右子树
        left_subtree = self._build_tree(left_data, features, depth + 1)
        right_subtree = self._build_tree(right_data, features, depth + 1)

        return {'feature': selected_feature, 'threshold': threshold,
                'left': left_subtree, 'right': right_subtree}
   
    # 创建叶子节点，包含数据集中出现最频繁的类别
    def _create_leaf_node(self, data):
        values, counts = np.unique(data.iloc[:, -1], return_counts=True)
        return {'class': values[np.argmax(counts)]}
    
    # 选择最佳特征和阈值进行分裂
    def _choose_best_feature(self, data, features):
        best_feature = None
        best_threshold = None
        best_info_gain_ratio = -1

        for feature in features:
            if data[feature].dtype == 'object':
                # 处理离散特征
                info_gain_ratio, threshold = self._calculate_info_gain_ratio(data, feature)
            else:
                # 处理连续特征
                info_gain_ratio, threshold = self._calculate_info_gain_ratio_continuous(data, feature)

            # 选择信息增益比最大的特征和相应的阈值
            if info_gain_ratio > best_info_gain_ratio:
                best_info_gain_ratio = info_gain_ratio
                best_feature = feature
                best_threshold = threshold

        return best_feature, best_threshold

    # 计算信息增益比
    def _calculate_info_gain_ratio(self, data, feature):

        # 计算原始数据的信息熵
        original_entropy = self._calculate_entropy(data.iloc[:, -1])

        # 计算按当前特征分裂后的条件熵
        values, counts = np.unique(data[feature], return_counts=True)
        weighted_entropy = 0
        for value, count in zip(values, counts):
            subset_indices = data[data[feature] == value].index
            subset_entropy = self._calculate_entropy(data.loc[subset_indices].iloc[:, -1])
            weighted_entropy += count / len(data) * subset_entropy

        # 计算信息增益
        information_gain = original_entropy - weighted_entropy

        # 计算分裂信息
        split_info = self._calculate_split_info(data[feature])

        # 避免分母为零
        if split_info == 0:
            return 0, 0

        # 计算信息增益比
        info_gain_ratio = information_gain / split_info

        return info_gain_ratio, None  # 对于离散特征，不需要阈值

    # 计算处理连续特征的信息增益比
    def _calculate_info_gain_ratio_continuous(self, data, feature):
        # 排序并获取唯一值
        unique_values = np.unique(data[feature])
        unique_values.sort()

        best_info_gain_ratio = -1
        best_threshold = None

        # 遍历每个可能的阈值,计算每个阈值下的信息增益比
        for i in range(len(unique_values) - 1):
            threshold = (unique_values[i] + unique_values[i + 1]) / 2
            info_gain_ratio, _ = self._calculate_info_gain_ratio_continuous_with_threshold(data, feature, threshold)

            # 选择最大的信息增益比和相应的阈值
            if info_gain_ratio > best_info_gain_ratio:
                best_info_gain_ratio = info_gain_ratio
                best_threshold = threshold

        return best_info_gain_ratio, best_threshold
    
    # 计算给定阈值下的信息增益比
    def _calculate_info_gain_ratio_continuous_with_threshold(self, data, feature, threshold):
        # 分割数据
        left_subset = data[data[feature] <= threshold]
        right_subset = data[data[feature] > threshold]

        # 计算原始数据的信息熵
        original_entropy = self._calculate_entropy(data.iloc[:, -1])

        # 计算按当前特征分裂后的条件熵
        left_weight = len(left_subset) / len(data)
        right_weight = len(right_subset) / len(data)

        left_subset_entropy = self._calculate_entropy(left_subset.iloc[:, -1])
        right_subset_entropy = self._calculate_entropy(right_subset.iloc[:, -1])

        weighted_entropy = left_weight * left_subset_entropy + right_weight * right_subset_entropy

        # 计算信息增益
        information_gain = original_entropy - weighted_entropy

        # 计算分裂信息
        split_info = self._calculate_split_info_continuous(data[feature], threshold)

        # 避免分母为零
        if split_info == 0:
            return 0, threshold

        # 计算信息增益比
        info_gain_ratio = information_gain / split_info

        return info_gain_ratio, threshold

    # 计算分裂信息（离散特征）
    def _calculate_split_info(self, x):
        _, counts = np.unique(x, return_counts=True)
        probabilities = counts / len(x)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

     # 计算分裂信息（连续特征）
    def _calculate_split_info_continuous(self, x, threshold):
        left_probability = np.sum(x <= threshold) / len(x)
        right_probability = 1 - left_probability
        return -left_probability * np.log2(left_probability + 1e-10) - right_probability * np.log2(right_probability + 1e-10)

    # 新样本的预测逻辑
    # 遍历决策树进行预测
    def predict(self, X):       
        predictions = []
        for _, sample in X.iterrows():
            predictions.append(self._traverse_tree(sample, self.tree))
        return np.array(predictions)

    # 递归遍历决策树，直到叶子节点
    def _traverse_tree(self, sample, node):
        if 'class' in node:
            return node['class']
        else:
            if sample[node['feature']] <= node['threshold']:
                return self._traverse_tree(sample, node['left'])
            else:
                return self._traverse_tree(sample, node['right'])

In [38]:
c45_tree = C45DecisionTree(min_samples_split=2, max_depth=None)
c45_tree.fit(X_train, y_train)

In [40]:
y_pred = c45_tree.predict(X_test)
accuracy_c4_5 = accuracy_score(y_test, y_pred)
print(f"C4.5 Classification Accuracy: {accuracy_c4_5:.2f}")

C4.5 Classification Accuracy: 0.60



### CART算法
- ID3和C4.5虽然在对训练样本集的学习中可以尽可能多的挖掘信息，但其生成的决策树分支较大，规模较大。为了简化决策树的规模，提高生成决策树的效率，就出现了根据**基尼指数**来选择的CART； 
- 对于给定的样本集合 ，其基尼指数为： $$ {Gini}(D)=1-\sum_{k=1}^{K}\left(\frac{\left|C_{k}\right|}{|D|}\right)^{2} $$
   其中$𝐶_𝑘$是𝐷中属于第𝑘类的样本子集，K是类的个数。
- 基尼系数的性质与信息熵一样：
   度量随机变量的不确定度的大小；基尼指数越⼩表示数据的纯度越高，反之其值越大，样本集合的不确定性也就越大。


In [45]:
class CARTDecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    # 构建决策树
    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
         # 若达到最大深度或样本数量小于等于阈值，则返回叶子节点
        if self.max_depth is not None and depth == self.max_depth or len(X) <= self.min_samples_split:
            return self._create_leaf_node(y)

         # 选择最佳特征和阈值进行分裂
        best_feature, best_threshold = self._choose_best_split(X, y)

        # 若无法找到合适的分裂点，则返回叶子节点
        if best_feature is None:
            return self._create_leaf_node(y)

        # 根据选择的特征和阈值进行分裂
        left_indices = X[best_feature] <= best_threshold
        right_indices = ~left_indices

        # 递归构建左右子树
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return {'feature': best_feature, 'threshold': best_threshold,
                'left': left_subtree, 'right': right_subtree}

    # 选择最佳特征和阈值进行分裂
    def _choose_best_split(self, X, y):
        best_feature = None
        best_threshold = None
        best_mse = float('inf')

         # 遍历所有特征
        for feature in X.columns:
            # 计算每个特征可能的分裂点及其均方误差
            thresholds, mse_values = self._calculate_mse_for_feature(X[feature], y)
            
             # 检查mse_values是否为空
            if len(mse_values) == 0:
                continue
                
            min_mse_index = np.argmin(mse_values)
            min_mse = mse_values[min_mse_index]

            # 更新最佳分裂点
            if min_mse < best_mse:
                best_mse = min_mse
                best_feature = feature
                best_threshold = thresholds[min_mse_index]

        return best_feature, best_threshold

    # 计算给定特征的每个可能分裂点及其均方误差
    def _calculate_mse_for_feature(self, feature, y):
        unique_values = np.unique(feature)
        thresholds = (unique_values[:-1] + unique_values[1:]) / 2

        mse_values = []

        for threshold in thresholds:
            left_indices = feature <= threshold
            right_indices = ~left_indices

            mse = self._calculate_mse(y[left_indices]) + self._calculate_mse(y[right_indices])
            mse_values.append(mse)

        return thresholds, mse_values

    # 计算均方误差
    def _calculate_mse(self, values):
        if len(values) == 0:
            return 0
        mean_value = np.mean(values)
        return np.mean((values - mean_value) ** 2)

    # 创建叶子节点，返回均值作为叶子节点的值
    def _create_leaf_node(self, y):
        return {'value': np.mean(y)}

     # 对输入数据进行预测
    def predict(self, X):
        predictions = []
        for _, sample in X.iterrows():
            predictions.append(self._traverse_tree(sample, self.tree))
        return np.array(predictions)

    # 递归遍历决策树，返回叶子节点的值
    def _traverse_tree(self, sample, node):
        if 'value' in node:
            return node['value']
        else:
            if sample[node['feature']] <= node['threshold']:
                return self._traverse_tree(sample, node['left'])
            else:
                return self._traverse_tree(sample, node['right'])

In [46]:
cart_tree = CARTDecisionTree(min_samples_split=2, max_depth=None)
cart_tree.fit(X_train, y_train)

In [48]:
y_pred = cart_tree.predict(X_test)
accuracy_cart = accuracy_score(y_test, y_pred)
print(f"CART Classification Accuracy: {accuracy_cart:.2f}")

CART Classification Accuracy: 0.60


### 高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

### 决策树的剪枝
- 决策树很容易出现**过拟合现象**。原因在于学习时完全考虑的是如何提⾼对训练数据的正确分类从⽽构建出过于复杂的决策树。
- 解决这个问题的方法称为**剪枝**，即对已生成的树进行简化。具体地，就是从已生成的树上裁剪掉⼀些子树或叶节点，并将其根节点或父节点作为新的叶节点。 
- 决策树的剪枝基本策略有**预剪枝 (Pre-Pruning)** 和 **后剪枝 (Post-Pruning)**
   - **预剪枝**：是根据⼀些原则**极早的停止树增长**，如树的深度达到用户所要的深度、节点中样本个数少于用户指定个数、不纯度指标下降的幅度小于用户指定的幅度等。 
   - **后剪枝**：是通过在完全生长的树上剪去分枝实现的，通过删除节点的分支来剪去树节点。是在生成决策树之后**自底向上**的对树中所有的非叶结点进⾏逐一考察 。

In [80]:
class PrunedCARTDecisionTree:
    def __init__(self, min_samples_split=2, max_depth=None, alpha=0.1):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.alpha = alpha
        self.tree = None

    def fit(self, X, y):
        features = X.columns
        data = pd.concat([X, y], axis=1)
        self.tree = self._build_tree(data, features)

    def _build_tree(self, data, features, depth=0):
        if len(data) <= self.min_samples_split or (self.max_depth is not None and depth == self.max_depth):
            return self._create_leaf_node(data)

        selected_feature, threshold = self._choose_best_split(data, features)

        left_data = data[data[selected_feature] <= threshold]
        right_data = data[data[selected_feature] > threshold]

        validation_accuracy_before_split = self._calculate_accuracy(left_data, right_data)

        left_leaf = self._create_leaf_node(left_data)
        right_leaf = self._create_leaf_node(right_data)

        validation_accuracy_after_split = self._calculate_accuracy(left_leaf, right_leaf)

        if validation_accuracy_after_split - validation_accuracy_before_split < self.alpha:
            return self._create_leaf_node(data)

        left_subtree = self._build_tree(left_data, features, depth + 1)
        right_subtree = self._build_tree(right_data, features, depth + 1)

        return {'feature': selected_feature, 'threshold': threshold,
                'left': left_subtree, 'right': right_subtree}

    def _choose_best_split(self, data, features):
        best_feature = None
        best_threshold = None
        best_mse = float('inf')

        for feature in features:
            thresholds, mse_values = self._calculate_mse_for_feature(data[feature], data.iloc[:, -1])
            min_mse_index = np.argmin(mse_values)
            min_mse = mse_values[min_mse_index]

            if min_mse < best_mse:
                best_mse = min_mse
                best_feature = feature
                best_threshold = thresholds[min_mse_index]

        return best_feature, best_threshold

    def _calculate_mse_for_feature(self, feature, y):
        thresholds = np.unique(feature)
        thresholds.sort()

        mse_values = []
        for threshold in thresholds:
            left_indices = feature <= threshold
            right_indices = ~left_indices
            mse = self._calculate_mse(y[left_indices]) + self._calculate_mse(y[right_indices])
            mse_values.append(mse)

        return thresholds, mse_values

    def _calculate_mse(self, values):
        if len(values) == 0:
            return 0
        mean_value = np.mean(values)
        return np.mean((values - mean_value) ** 2)

    def _create_leaf_node(self, data):
        values, counts = np.unique(data.iloc[:, -1], return_counts=True)
        return {'class': values[np.argmax(counts)]}

    def _calculate_accuracy(self, left_data, right_data):
        return len(left_data) / (len(left_data) + len(right_data))

    def predict(self, X):
        predictions = []
        for _, sample in X.iterrows():
            predictions.append(self._traverse_tree(sample, self.tree))
        return np.array(predictions)

    def _traverse_tree(self, sample, node):
        if 'class' in node:
            return node['class']
        else:
            if sample[node['feature']] <= node['threshold']:
                return self._traverse_tree(sample, node['left'])
            else:
                return self._traverse_tree(sample, node['right'])

计算分裂前左右子树的准确率，然后创建左右子树的叶子节点，再计算分裂后的准确率。如果分裂后的准确率减去分裂前的准确率小于 self.alpha，则选择不进行分裂，而是直接返回一个包含整个数据的叶子节点，

In [97]:
pruned_cart_tree = PrunedCARTDecisionTree(min_samples_split=2, max_depth=None, alpha=0.1)
pruned_cart_tree.fit(X_train, y_train)

# tree = C45DecisionTree(min_samples_split=2, max_depth=None)
# tree.fit(X_train, y_train)

In [99]:
# 预测并输出分类精度
y_pred = pruned_cart_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Pruned CART Classification Accuracy: {accuracy:.2f}")

Pruned CART Classification Accuracy: 0.40


In [94]:
class PrunedC45DecisionTree:
    
    """
    min_samples_split: int, optional (default=2)
          节点分裂的最小样本数
    max_depth: int or None, optional (default=None)
          树的最大深度。如果为None，则节点会一直分裂，直到每个叶子节点的样本数小于min_samples_split     
    tree: dict
          存储构建好的决策树的字典    
    """
    
    def __init__(self, min_samples_split=2, max_depth=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.tree = None

    # 计算信息熵
    def _calculate_entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))
    
    # 根据训练数据拟合C4.5决策树
    def fit(self, X, y, n_folds=5):
        features = X.columns
        data = pd.concat([X, y], axis=1)

        kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

        for train_index, val_index in kf.split(data):
            train_data, val_data = data.iloc[train_index], data.iloc[val_index]
            
            self.tree = self._build_tree(train_data, features, validation_data=val_data)
            
            # 检查在剪枝后 self.tree 是否为 None
            if self.tree is not None:
                # 如果 self.tree 不是 None，则检查剪枝后的准确率
                pruned_tree = self._prune_tree(self.tree, val_data)
                if pruned_tree is not None:
                    self.tree = pruned_tree

    # 递归构建C4.5决策树
    def _build_tree(self, data, features, depth=0, validation_data=None):
        # 若样本数量小于等于阈值或深度达到最大深度，则返回叶子节点
        if len(data) <= self.min_samples_split or (self.max_depth is not None and depth == self.max_depth):
            return self._create_leaf_node(data)

        # 选择最佳特征和阈值进行分裂
        selected_feature, threshold = self._choose_best_feature(data, features)

        # 根据选择的特征和阈值进行分裂
        left_data = data[data[selected_feature] <= threshold]
        right_data = data[data[selected_feature] > threshold]

        # 递归构建左右子树
        left_subtree = self._build_tree(left_data, features, depth + 1)
        right_subtree = self._build_tree(right_data, features, depth + 1)

        # 构建完整的树
        self.tree = {'feature': selected_feature, 'threshold': threshold, 'left': left_subtree, 'right': right_subtree}

         # 进行后剪枝
        if validation_data is not None:
            pruned_tree = self._prune_tree(self.tree, validation_data)
            if pruned_tree is not None:
                self.tree = pruned_tree
 
        return self.tree
   
    def _prune_tree(self, tree, validation_data):
        if 'left' in tree and 'right' in tree:
            left_subtree = tree['left']
            right_subtree = tree['right']

            if 'class' not in left_subtree and 'class' not in right_subtree:
                # 如果左右子树都不是叶子节点，则递归剪枝
                tree['left'] = self._prune_tree(left_subtree, validation_data)
                tree['right'] = self._prune_tree(right_subtree, validation_data)

                # 如果剪枝后的准确率更高，则合并为叶子节点
                before_pruning_accuracy = self._calculate_accuracy(validation_data, tree)
                tree['class'] = self._majority_class(validation_data)
                after_pruning_accuracy = self._calculate_accuracy(validation_data, tree)

                if after_pruning_accuracy >= before_pruning_accuracy:
                    # 如果准确率提高或保持不变，则返回叶子节点
                    return {'class': tree['class']}

        return tree
    
    def _calculate_accuracy(self, data, tree):
        predictions = self.predict(data.iloc[:, :-1])
        actual_labels = data.iloc[:, -1].to_numpy()
        accuracy = np.mean(predictions == actual_labels)
        return accuracy
    
    def _majority_class(self, data):
        values, counts = np.unique(data.iloc[:, -1], return_counts=True)
        return values[np.argmax(counts)]
    
    # 创建叶子节点，包含数据集中出现最频繁的类别
    def _create_leaf_node(self, data):
        values, counts = np.unique(data.iloc[:, -1], return_counts=True)
        return {'class': values[np.argmax(counts)]}
    
    # 选择最佳特征和阈值进行分裂
    def _choose_best_feature(self, data, features):
        best_feature = None
        best_threshold = None
        best_info_gain_ratio = -1

        for feature in features:
            if data[feature].dtype == 'object':
                # 处理离散特征
                info_gain_ratio, threshold = self._calculate_info_gain_ratio(data, feature)
            else:
                # 处理连续特征
                info_gain_ratio, threshold = self._calculate_info_gain_ratio_continuous(data, feature)

            # 选择信息增益比最大的特征和相应的阈值
            if info_gain_ratio > best_info_gain_ratio:
                best_info_gain_ratio = info_gain_ratio
                best_feature = feature
                best_threshold = threshold

        return best_feature, best_threshold

    # 计算信息增益比
    def _calculate_info_gain_ratio(self, data, feature):

        # 计算原始数据的信息熵
        original_entropy = self._calculate_entropy(data.iloc[:, -1])

        # 计算按当前特征分裂后的条件熵
        values, counts = np.unique(data[feature], return_counts=True)
        weighted_entropy = 0
        for value, count in zip(values, counts):
            subset_indices = data[data[feature] == value].index
            subset_entropy = self._calculate_entropy(data.loc[subset_indices].iloc[:, -1])
            weighted_entropy += count / len(data) * subset_entropy

        # 计算信息增益
        information_gain = original_entropy - weighted_entropy

        # 计算分裂信息
        split_info = self._calculate_split_info(data[feature])

        # 避免分母为零
        if split_info == 0:
            return 0, 0

        # 计算信息增益比
        info_gain_ratio = information_gain / split_info

        return info_gain_ratio, None  # 对于离散特征，不需要阈值

    # 计算处理连续特征的信息增益比
    def _calculate_info_gain_ratio_continuous(self, data, feature):
        # 排序并获取唯一值
        unique_values = np.unique(data[feature])
        unique_values.sort()

        best_info_gain_ratio = -1
        best_threshold = None

        # 遍历每个可能的阈值,计算每个阈值下的信息增益比
        for i in range(len(unique_values) - 1):
            threshold = (unique_values[i] + unique_values[i + 1]) / 2
            info_gain_ratio, _ = self._calculate_info_gain_ratio_continuous_with_threshold(data, feature, threshold)

            # 选择最大的信息增益比和相应的阈值
            if info_gain_ratio > best_info_gain_ratio:
                best_info_gain_ratio = info_gain_ratio
                best_threshold = threshold

        return best_info_gain_ratio, best_threshold
    
    # 计算给定阈值下的信息增益比
    def _calculate_info_gain_ratio_continuous_with_threshold(self, data, feature, threshold):
        # 分割数据
        left_subset = data[data[feature] <= threshold]
        right_subset = data[data[feature] > threshold]

        # 计算原始数据的信息熵
        original_entropy = self._calculate_entropy(data.iloc[:, -1])

        # 计算按当前特征分裂后的条件熵
        left_weight = len(left_subset) / len(data)
        right_weight = len(right_subset) / len(data)

        left_subset_entropy = self._calculate_entropy(left_subset.iloc[:, -1])
        right_subset_entropy = self._calculate_entropy(right_subset.iloc[:, -1])

        weighted_entropy = left_weight * left_subset_entropy + right_weight * right_subset_entropy

        # 计算信息增益
        information_gain = original_entropy - weighted_entropy

        # 计算分裂信息
        split_info = self._calculate_split_info_continuous(data[feature], threshold)

        # 避免分母为零
        if split_info == 0:
            return 0, threshold

        # 计算信息增益比
        info_gain_ratio = information_gain / split_info

        return info_gain_ratio, threshold

    # 计算分裂信息（离散特征）
    def _calculate_split_info(self, x):
        _, counts = np.unique(x, return_counts=True)
        probabilities = counts / len(x)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

     # 计算分裂信息（连续特征）
    def _calculate_split_info_continuous(self, x, threshold):
        left_probability = np.sum(x <= threshold) / len(x)
        right_probability = 1 - left_probability
        return -left_probability * np.log2(left_probability + 1e-10) - right_probability * np.log2(right_probability + 1e-10)

    # 新样本的预测逻辑
    # 遍历决策树进行预测
    def predict(self, X):       
        predictions = []
        for _, sample in X.iterrows():
            predictions.append(self._traverse_tree(sample, self.tree))
        return np.array(predictions)

    # 递归遍历决策树，直到叶子节点
    def _traverse_tree(self, sample, node):
        if 'class' in node:
            return node['class']
        else:
            if sample[node['feature']] <= node['threshold']:
                return self._traverse_tree(sample, node['left'])
            else:
                return self._traverse_tree(sample, node['right'])

In [95]:
pruned_c45_tree = PrunedC45DecisionTree(min_samples_split=2, max_depth=None)
pruned_c45_tree.fit(X_train, y_train)

In [96]:
y_pred = pruned_c45_tree.predict(X_test)
accuracy_pruned_c45 = accuracy_score(y_test, y_pred)
print(f"C4.5 Classification Accuracy: {accuracy_pruned_c45:.2f}")

C4.5 Classification Accuracy: 0.60


### 决策树的剪枝 
- 决策树的剪枝基本策略有**预剪枝 (Pre-Pruning)** 和 **后剪枝 (Post-Pruning)**
   - **预剪枝**：是根据⼀些原则**极早的停止树增长**，如树的深度达到用户所要的深度、节点中样本个数少于用户指定个数、不纯度指标下降的幅度小于用户指定的幅度等。 
   - **后剪枝**：是通过在完全生长的树上剪去分枝实现的，通过删除节点的分支来剪去树节点。是在生成决策树之后**自底向上**的对树中所有的非叶结点进⾏逐一考察 。

预剪枝
- 预剪枝能够减少决策树的规模，因为它在构建树的过程中就进行了剪枝。这可以减少模型的复杂性，降低过拟合的风险.
- 预剪枝有助于防止决策树在训练数据上过分拟合，但过早停止划分可能导致模型无法学习复杂的模式，增加了欠拟合的风险

后剪枝
- 后剪枝在构建完整个树后进行，相比于预剪枝，它更灵活，可以更好地适应数据的复杂性。后剪枝通过剪枝决策树的一部分，有助于提高模型的泛化性能
- 通过减少树的复杂性，有助于防止过拟合。它可以根据验证集或交叉验证的性能来确定哪些节点需要剪枝
- 